In [1]:
!pip uninstall apache-beam -y && pip install -U pandas polars pyarrow narwhals

Found existing installation: apache-beam 2.46.0
Uninstalling apache-beam-2.46.0:
  Successfully uninstalled apache-beam-2.46.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 15.0.2
    Uninstalling pyarrow-15.0.2:
      Successfully uninstalled pyarrow-15.0.2
  Attempting uninstall: polars
    Found existing installation: polars 0.20.15
    Uninstalling polars-0.20.15:
      Successfully uninstalled polars-0.20.15
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.1
    Uninstalling pandas-2.2.1:
      Successfully uninstalled pandas-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages th

In [2]:
from typing import Any
from datetime import date

import narwhals as nw

import pandas as pd
import polars as pl

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True

In [3]:
Q_NUM = 21

IO_FUNCS = {
    'pandas': lambda x: pd.read_parquet(x, engine='pyarrow'),
    'pandas[pyarrow]': lambda x: pd.read_parquet(x, engine='pyarrow', dtype_backend='pyarrow'),
    'polars[eager]': lambda x: pl.read_parquet(x),
    'polars[lazy]': lambda x: pl.scan_parquet(x),
}

In [4]:
def q21(
    lineitem_raw: Any,
    nation_raw: Any,
    orders_raw: Any,
    supplier_raw: Any,
) -> Any:
    
    lineitem = nw.from_native(lineitem_raw)
    nation = nw.from_native(nation_raw)
    orders = nw.from_native(orders_raw)
    supplier = nw.from_native(supplier_raw)
    
    var1 = "SAUDI ARABIA"
    
    
    q1 = (
        lineitem.group_by("l_orderkey")
#         .agg(nw.col("l_suppkey").len().alias("n_supp_by_order"))
        .agg(nw.len().alias("n_supp_by_order"))
        .filter(nw.col("n_supp_by_order") > 1)
        .join(
            lineitem.filter(nw.col("l_receiptdate") > nw.col("l_commitdate")),
#             on="l_orderkey",
            left_on="l_orderkey", right_on="l_orderkey",
        )
    )

    q_final = (
        q1.group_by("l_orderkey")
#         .agg(nw.col("l_suppkey").len().alias("n_supp_by_order"))
        .agg(nw.len().alias("n_supp_by_order"))
        .join(q1, left_on="l_orderkey", right_on="l_orderkey"
              #on="l_orderkey"
             )
        .join(supplier, left_on="l_suppkey", right_on="s_suppkey")
        .join(nation, left_on="s_nationkey", right_on="n_nationkey")
        .join(orders, left_on="l_orderkey", right_on="o_orderkey")
        .filter(nw.col("n_supp_by_order") == 1)
        .filter(nw.col("n_name") == var1)
        .filter(nw.col("o_orderstatus") == "F")
        .group_by("s_name")
        .agg(nw.len().alias("numwait"))
        .sort(by=["numwait", "s_name"], descending=[True, False])
        .head(100)
    )
    return nw.to_native(q_final)

In [5]:
dir_ = "/kaggle/input/tpc-h-data-parquet-s-2/"

lineitem = dir_ + 'lineitem.parquet'
nation = dir_ + 'nation.parquet'
orders = dir_ + 'orders.parquet'
supplier = dir_ + 'supplier.parquet'

In [6]:
results = {}

## pandas via Narwhals

In [7]:
tool = 'pandas'
fn = IO_FUNCS[tool]

lineitem_raw, nation_raw, orders_raw, supplier_raw = fn(lineitem), fn(nation), fn(orders), fn(supplier)

timings = %timeit -o q21(lineitem_raw, nation_raw, orders_raw, supplier_raw)
results[tool] = timings.all_runs

18.5 s ± 5.39 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## pandas, pyarrow dtypes, via Narwhals

In [8]:
tool = 'pandas[pyarrow]'
fn = IO_FUNCS[tool]
lineitem_raw, nation_raw, orders_raw, supplier_raw = fn(lineitem), fn(nation), fn(orders), fn(supplier)

timings = %timeit -o q21(lineitem_raw, nation_raw, orders_raw, supplier_raw)
results[tool] = timings.all_runs

16.7 s ± 368 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Polars read_parquet

In [9]:
tool = 'polars[eager]'
fn = IO_FUNCS[tool]

lineitem_raw, nation_raw, orders_raw, supplier_raw = fn(lineitem), fn(nation), fn(orders), fn(supplier)
timings = %timeit -o q21(lineitem_raw, nation_raw, orders_raw, supplier_raw)
results[tool] = timings.all_runs

10.7 s ± 157 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Polars scan_parquet

In [10]:
tool = 'polars[lazy]'
fn = IO_FUNCS[tool]

lineitem_raw, nation_raw, orders_raw, supplier_raw = fn(lineitem), fn(nation), fn(orders), fn(supplier)
timings = %timeit -o q21(lineitem_raw, nation_raw, orders_raw, supplier_raw).collect()
results[tool] = timings.all_runs

2.92 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Save

In [11]:
import json
with open('results.json', 'w') as fd:
    json.dump(results, fd)

In [12]:
from pprint import pprint

pprint(results)

{'pandas': [14.536162291999972,
            28.61268175500004,
            25.131280856000103,
            15.984822283000085,
            15.986703961000103,
            14.77401841599999,
            14.613077059000034],
 'pandas[pyarrow]': [16.43704511399983,
                     17.436027875000036,
                     16.616333898999983,
                     16.929478356000118,
                     16.75605714900007,
                     16.174442130999978,
                     16.778590352000037],
 'polars[eager]': [10.55738047900013,
                   10.522792472999981,
                   10.959035934999974,
                   10.62960384400003,
                   10.635097036999923,
                   10.881635524000103,
                   10.81637764800007],
 'polars[lazy]': [2.9100079499999083,
                  2.8206382559999383,
                  2.8398498980000113,
                  2.848305809000067,
                  3.236310110999966,
                  2.933306013999